<a href="https://colab.research.google.com/github/chugcha/hw_proga/blob/main/hw_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Домашнее задание 2**

# Загрузка книги и всего остального

In [121]:
!pip install pymystem3 --q
!pip install nltk --q
!pip install pymorphy3 --q
!pip install spacy --q
!pip install typing_extensions --upgrade --q
!python -m spacy download ru_core_news_sm --q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 95.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [122]:
!wget https://raw.githubusercontent.com/chugcha/hw_proga/refs/heads/main/bazhov1tom.txt
# Это файл из моего репозитория на гитхабе

with open('bazhov1tom.txt', 'r') as file:
   book = file.read()

--2025-10-21 22:17:22--  https://raw.githubusercontent.com/chugcha/hw_proga/refs/heads/main/bazhov1tom.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917244 (896K) [text/plain]
Saving to: ‘bazhov1tom.txt’

bazhov1tom.txt      100%[===================>] 895.75K  --.-KB/s    in 0.05s   

2025-10-21 22:17:23 (18.5 MB/s) - ‘bazhov1tom.txt’ saved [917244/917244]



# NLTK + Pymorphy

In [123]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt_tab')

book_tokens = word_tokenize(book, language='russian')
sw = stopwords.words('russian')

dirty_words = [w.lower() for w in book_tokens if w.isalpha()]
clean_words = [w for w in dirty_words if w not in sw]
# теперь в clean_words остались только все не-стоп-слова

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [126]:
import json
from pymorphy3 import MorphAnalyzer
from pprint import pprint

morph = MorphAnalyzer()

ana_w = [morph.parse(word) for word in clean_words]
# здесь записаны все возможные разборы слов из текста

with open('analysed_tokens_pymorphy.jsonl', 'w', encoding='utf-8') as f:
    for i in ana_w:
        token_data = {"word": i[0].word, "lemma": i[0].normalized.word, "pos": i[0].tag.POS}
        f.write(json.dumps(token_data) + '\n')

# теперь всё есть в json файле, смотрите:

with open('analysed_tokens_pymorphy.jsonl', 'r', encoding='utf-8') as k:
    for i in range(6):
        print(json.loads(k.readline()))

{'word': 'бажов', 'lemma': 'баж', 'pos': 'NOUN'}
{'word': 'павел', 'lemma': 'павел', 'pos': 'NOUN'}
{'word': 'собрание', 'lemma': 'собрание', 'pos': 'NOUN'}
{'word': 'сочинений', 'lemma': 'сочинение', 'pos': 'NOUN'}
{'word': 'трёх', 'lemma': 'три', 'pos': 'NUMR'}
{'word': 'томах', 'lemma': 'том', 'pos': 'NOUN'}


In [127]:
#заносим информацию по каждой лемме в словарик для конвертирования в датафрейм
data_dict = {'lemmas': [], 'pos': [], 'frequency': []}

for i in ana_w:
  if i[0].normalized.word not in data_dict['lemmas']: # добавляем уникальную лемму
      data_dict['lemmas'].append(i[0].normalized.word)
      data_dict['pos'].append(i[0].normalized.tag.POS)
      data_dict['frequency'].append(1)

  else:
      # оказалось, бывает, что одну лемму определяют как разные части речи
      if data_dict['pos'][data_dict['lemmas'].index(i[0].normalized.word)] != i[0].normalized.tag.POS:
          same_lemmas_POS = []
          same_lemmas = [j for j, item in enumerate(data_dict['lemmas']) if item == i[0].normalized.word]
          for m in same_lemmas:
              same_lemmas_POS.append(data_dict['pos'][m])
          if i[0].normalized.tag.POS not in same_lemmas_POS:
              data_dict['lemmas'].append(i[0].normalized.word)
              data_dict['pos'].append(i[0].normalized.tag.POS)
              data_dict['frequency'].append(1)
      else:
          data_dict['frequency'][data_dict['lemmas'].index(i[0].normalized.word)] += 1
# это на всякий случай, проверить, что работает
print('Всего лексем:', len(data_dict['lemmas']))
print('Абсолютная частота вхождений лексемы "кошка":', data_dict['frequency'][data_dict['lemmas'].index('кошка')])


Всего лексем: 7836
Абсолютная частота вхождений лексемы "кошка": 37


In [128]:
import pandas as pd
bazhov_df = pd.DataFrame(data_dict)
print(bazhov_df.info())
print('\n')
#видно, что у некоторых слов не хватает каких-то значений POS. Выведем их
words_without_pos = bazhov_df[bazhov_df['pos'].isna()]
print(f"Слова без части речи: {len(words_without_pos)}")
print(words_without_pos['lemmas'].to_list())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7836 entries, 0 to 7835
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lemmas     7836 non-null   object
 1   pos        7820 non-null   object
 2   frequency  7836 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 183.8+ KB
None


Слова без части речи: 16
['постеля', 'пировля', 'ина', 'гож', 'дедко', 'полишку', 'задохся', 'ремки', 'нелзя', 'еси', 'заделья', 'деревнешка', 'гач', 'кго', 'деревнл', 'эку']


In [129]:
sorted_df = bazhov_df.sort_values('frequency', ascending=False)
print(sorted_df)

           lemmas   pos  frequency
41          стать  INFN        530
122      говорить  INFN        509
131           это  PRCL        442
295           тот  ADJF        375
192          дело  NOUN        311
...           ...   ...        ...
7818      площадь  NOUN          1
7817  понаторкать  INFN          1
7816      знаться  INFN          1
7815   замкнуться  INFN          1
7814     остатный  ADJF          1

[7836 rows x 3 columns]


In [130]:
import csv

sorted_df.to_csv('pymorphy_frequency.csv')

# Spacy

In [131]:
import spacy
from spacy.lang.ru.examples import sentences


nlp = spacy.load('ru_core_news_sm')

In [134]:
# ну если нельзя, то очистить можно так:

cleaned_book = []

doc_sad = nlp(book)
for token in doc_sad:
  if not token.is_stop and not token.is_punct and not token.is_space:
      cleaned_book.append(token.text)
doc_happy = nlp(' '.join(clean_words))

In [135]:
print(cleaned_book[:10])

['Бажов', 'Павел', 'Собрание', 'сочинений', 'трех', 'Сказки', '1', 'МЕДНОЙ', 'ГОРЫ', 'ХОЗЯЙКА']


In [138]:
# так же, как было с pymorphy и nltk
with open('analysed_tokens_spacy.jsonl', 'w', encoding='utf-8') as f:
    for token in doc_happy:
        token_data = {"word": token.text, "lemma": token.lemma_, "pos": token.pos_}
        f.write(json.dumps(token_data) + '\n')

# вроде бы всё на месте
with open('analysed_tokens_spacy.jsonl', 'r', encoding='utf-8') as k:
    for i in range(6):
        print(json.loads(k.readline()))

{'word': 'бажов', 'lemma': 'бажов', 'pos': 'NOUN'}
{'word': 'павел', 'lemma': 'павел', 'pos': 'VERB'}
{'word': 'собрание', 'lemma': 'собрание', 'pos': 'NOUN'}
{'word': 'сочинений', 'lemma': 'сочинение', 'pos': 'NOUN'}
{'word': 'трех', 'lemma': 'три', 'pos': 'NUM'}
{'word': 'томах', 'lemma': 'том', 'pos': 'NOUN'}


In [139]:
# делаем всё аналогично с pymorph
data_dict_sp = {'lemmas': [], 'pos': [], 'frequency': []}

for token in doc_happy:
  if token.lemma_ not in data_dict_sp['lemmas']: # добавляем уникальную лемму
      data_dict_sp['lemmas'].append(token.lemma_)
      data_dict_sp['pos'].append(token.pos_)
      data_dict_sp['frequency'].append(1)

  else:
      # оказалось, бывает, что одну лемму определяют как разные части речи
      if data_dict_sp['pos'][data_dict_sp['lemmas'].index(token.lemma_)] != token.pos_:
          same_lemmas_POS = []
          same_lemmas = [j for j, item in enumerate(data_dict_sp['lemmas']) if item == token.lemma_]
          for m in same_lemmas:
              same_lemmas_POS.append(data_dict_sp['pos'][m])
          if token.pos_ not in same_lemmas_POS:
              data_dict_sp['lemmas'].append(token.lemma_)
              data_dict_sp['pos'].append(token.pos_)
              data_dict_sp['frequency'].append(1)
      else:
          data_dict_sp['frequency'][data_dict_sp['lemmas'].index(token.lemma_)] += 1
# это на всякий случай, проверить, что работает
print('Всего лексем:', len(data_dict_sp['lemmas']))
print('Абсолютная частота вхождений лексемы "кошка":', data_dict_sp['frequency'][data_dict_sp['lemmas'].index('кошка')])


Всего лексем: 11623
Абсолютная частота вхождений лексемы "кошка": 37


In [140]:
bazhov_sp = pd.DataFrame(data_dict_sp)
print(bazhov_sp.info(), '\n')

sorted_df_sp = bazhov_sp.sort_values('frequency', ascending=False)
print('А вот и частотность: \n')
print(sorted_df_sp)

sorted_df_sp.to_csv('spacy_frequency.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11623 entries, 0 to 11622
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lemmas     11623 non-null  object
 1   pos        11623 non-null  object
 2   frequency  11623 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 272.5+ KB
None 

А вот и частотность: 

            lemmas   pos  frequency
42           стать  VERB        504
125       говорить  VERB        499
311            тот   DET        353
134            это  PRON        338
107          место  NOUN        290
...            ...   ...        ...
20     праздничный  VERB          1
11607       угадаю  VERB          1
11608    болботать   ADV          1
11609      одинова   ADJ          1
11594   огромадная   ADJ          1

[11623 rows x 3 columns]
